Import packages and processed run metrics

In [83]:
# packages
import re
import numpy as np
import pandas as pd

# control
project_names = [
    'decay--sst2',
    'decay--pubmed',
    'decay--cifar--alpha5.0',
    'decay--cifar--alpha0.5',
    'decay--cifar--alpha0.1'
]
validation_metric = 'Results/val_acc'

# read each data set
df = []
for project_name in project_names:
    temp = pd.read_csv(f'{project_name}.csv')
    print(temp.shape)
    df.append(temp)

df = pd.concat(df, axis=0, ignore_index=True)
print()
print('all runs:', df.shape)

(1201, 36)
(780, 28)
(560, 27)
(520, 27)
(520, 27)

all runs: (3581, 37)


Write function to return summaries of run's metric performance

In [84]:
def process_run_metrics(df, method='exact'):
    pass

    # get best runs for each group
    idx = df.groupby(['dataset', 'method', 'finetune']) \
        [validation_metric].idxmax()  # return index of max validation metric
    filtered_df = df.loc[idx]


    ## Process metrics and get top runs for each
    # are large or small metric values are desirable?
    descending_metrics = [name for name in filtered_df.columns if re.search('Results.*test', name)]
    ascending_metrics = [
        descending_metrics.pop(descending_metrics.index(name))
        for name in descending_metrics
        if re.search('std', name)
    ]
    print('metrics:')
    print('\t', descending_metrics)
    print('\t', ascending_metrics)

    # convert metrics to numeric
    #filtered_df[descending_metrics + ascending_metrics] = filtered_df[descending_metrics + ascending_metrics].apply(pd.to_numeric)

    # all non-metrics columns are used to identify the experimental run
    filtered_runs = filtered_df[[
        name for name in filtered_df.columns
        if name not in descending_metrics + ascending_metrics
    ]]

    # rank the metrics
    ranked_descending = filtered_df.groupby(['dataset'])[descending_metrics].rank(
        method='min',
        ascending=False
    )
    ranked_ascending = filtered_df.groupby(['dataset'])[ascending_metrics].rank(
        method='min',
        ascending=True
    )

    # combine and sort the ranked_metrics
    ranked_metrics = pd.concat([ranked_descending, ranked_ascending], axis=1)
    ranked_metrics = ranked_metrics[sorted(ranked_metrics.columns)]
    filtered_ranks = filtered_runs.join(ranked_metrics)


    ## Manipulate rank data to be summarized by runs and metrics
    # convert to long format
    # filter to top ranks
    long_filtered_ranks = pd.melt(filtered_ranks, id_vars=filtered_runs.columns, var_name='metric')
    top_filtered_metrics = long_filtered_ranks.loc[long_filtered_ranks.value <= 3].copy()  # top 3 runs

    # compute rank summaries to understand what runs are top overall
    top_filtered_metrics['rank_one_ind'] = (top_filtered_metrics.value == 1)
    top_filtered_metrics['rank_two_ind'] = (top_filtered_metrics.value == 2)
    top_filtered_metrics['rank_three_ind'] = (top_filtered_metrics.value == 3)
    top_filtered_metrics.replace(False, pd.NA, inplace=True)

    # summarized metric ranks for run type
    rank_summary_columns = ['rank_one_ind', 'rank_two_ind', 'rank_three_ind', 'value']
    #id_columns = ['dataset', 'method', 'finetune']
    id_columns = ['method', 'finetune']
    run_summary = top_filtered_metrics.groupby(id_columns)[rank_summary_columns].count()

    # summarize metric ranks for metric choice
    metric_summary = top_filtered_metrics.loc[top_filtered_metrics.method == method]
    metric_summary = metric_summary.sort_values(by='metric').groupby('metric')[rank_summary_columns].count()

    return (
        run_summary,
        metric_summary,
        filtered_runs
    )


Filter data to runs of interest
Apply summary function to filtered dataset

In [85]:
subset_df = df

# row (run) filtering
subset_df = subset_df.loc[(df.n_epochs < 6)]
betas = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
subset_df = subset_df.loc[[
    True if beta in betas else False
    for beta in subset_df.beta
    ] | (subset_df.method != 'exact')
]

# column (metric) filtering
subset_df = subset_df[[
    name for name in subset_df.columns
    if not re.search('f1', name)
       and not re.search('loss', name)
]]

(my_run_summary, my_metric_summary, temp) = process_run_metrics(subset_df, method='exact')

metrics:
	 ['Results/test_acc', 'Results/test_acc_bottom_decile', 'Results_unseen/test_acc', 'Results_unseen/test_acc_bottom_decile', 'Results_weighted/test_acc', 'Results_weighted_unseen/test_acc']
	 ['Results/test_acc_std', 'Results_unseen/test_acc_std']


View processed summaries

In [86]:
print(my_run_summary.shape)
my_run_summary.sort_values(by='value', ascending=False)

(12, 4)


,,rank_one_ind,rank_two_ind,rank_three_ind,value
method,finetune,,,,
exact,0,14,8,4,26
fedavg,0,11,9,3,23
fedem,0,6,4,13,23
pfedme,0,8,3,3,14
exact,1,4,3,0,7
fedbn,1,1,3,3,7
fedem,1,2,1,2,5
pfedme,1,2,0,2,4
fedavg,1,0,1,1,2


In [87]:
print(my_metric_summary.shape)
sort_by = ['value', 'rank_one_ind', 'rank_two_ind', 'rank_three_ind']
my_metric_summary.sort_values(by=sort_by, ascending=False)

(8, 4)


,rank_one_ind,rank_two_ind,rank_three_ind,value
metric,,,,
Results_unseen/test_acc,3,1,1,5
Results_weighted_unseen/test_acc,3,1,1,5
Results/test_acc_bottom_decile,2,3,0,5
Results/test_acc_std,4,0,0,4
Results/test_acc,2,2,0,4
Results_unseen/test_acc_std,1,2,1,4
Results_unseen/test_acc_bottom_decile,2,0,1,3
Results_weighted/test_acc,1,2,0,3


In [88]:
temp.head()

,Unnamed: 0,method,finetune,dataset,n_epochs,batch_size,lr,beta,regular_weight,K,finetune_lr,Results/val_acc,alpha
3399,338,ditto,0,cifar--alpha0.1,3,NaN,0.50,NaN,0.5,NaN,NaN,0.487736,0.1
3548,487,exact,0,cifar--alpha0.1,3,NaN,0.10,0.2,NaN,NaN,NaN,0.566214,0.1
3373,312,fedavg,0,cifar--alpha0.1,3,NaN,0.05,NaN,NaN,NaN,NaN,0.553132,0.1
3351,290,fedbn,0,cifar--alpha0.1,3,NaN,0.10,NaN,NaN,NaN,NaN,0.407272,0.1
3353,292,fedbn,1,cifar--alpha0.1,3,NaN,0.01,NaN,NaN,NaN,NaN,0.349454,0.1
